In [0]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www-eu.apache.org/dist/spark/spark-2.3.3/spark-2.3.3-bin-hadoop2.7.tgz
!tar xf spark-2.3.3-bin-hadoop2.7.tgz
!pip install -q findspark

In [0]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.3.3-bin-hadoop2.7"

In [0]:
import findspark
findspark.init()
from pyspark.sql import SparkSession

In [0]:
spark = SparkSession.builder.appName('mytree').getOrCreate()

In [0]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import (RandomForestClassifier,GBTClassifier,
                                       DecisionTreeClassifier)

In [0]:
data = spark.read.format('libsvm').load('sample_libsvm_data.txt')

In [8]:
data.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(692,[127,128,129...|
|  1.0|(692,[158,159,160...|
|  1.0|(692,[124,125,126...|
|  1.0|(692,[152,153,154...|
|  1.0|(692,[151,152,153...|
|  0.0|(692,[129,130,131...|
|  1.0|(692,[158,159,160...|
|  1.0|(692,[99,100,101,...|
|  0.0|(692,[154,155,156...|
|  0.0|(692,[127,128,129...|
|  1.0|(692,[154,155,156...|
|  0.0|(692,[153,154,155...|
|  0.0|(692,[151,152,153...|
|  1.0|(692,[129,130,131...|
|  0.0|(692,[154,155,156...|
|  1.0|(692,[150,151,152...|
|  0.0|(692,[124,125,126...|
|  0.0|(692,[152,153,154...|
|  1.0|(692,[97,98,99,12...|
|  1.0|(692,[124,125,126...|
+-----+--------------------+
only showing top 20 rows



In [0]:
train_data,test_data = data.randomSplit([0.7,0.3])

In [0]:
dtc = DecisionTreeClassifier()
rfc = RandomForestClassifier(numTrees=100)
gbt = GBTClassifier()

In [0]:
dtc_model = dtc.fit(train_data)
rfc_model = rfc.fit(train_data)
gbt_model = gbt.fit(train_data)

In [0]:
dtc_preds = dtc_model.transform(test_data)
rfc_preds = rfc_model.transform(test_data)
gbt_preds = gbt_model.transform(test_data)

In [0]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [0]:
acc_eval = MulticlassClassificationEvaluator(metricName='accuracy')

In [17]:
print('DTC ACCURACY')
acc_eval.evaluate(dtc_preds)

DTC ACCURACY


0.9444444444444444

In [18]:
print('RFC ACCURACY')
acc_eval.evaluate(rfc_preds)

RFC ACCURACY


1.0

In [19]:
print('GBT ACCURACY')
acc_eval.evaluate(gbt_preds)

GBT ACCURACY


0.9444444444444444

### Code Along

In [0]:
spark = SparkSession.builder.appName('tree').getOrCreate()

In [0]:
data = spark.read.csv("College.csv",inferSchema=True,header=True)

In [22]:
data.printSchema()

root
 |-- School: string (nullable = true)
 |-- Private: string (nullable = true)
 |-- Apps: integer (nullable = true)
 |-- Accept: integer (nullable = true)
 |-- Enroll: integer (nullable = true)
 |-- Top10perc: integer (nullable = true)
 |-- Top25perc: integer (nullable = true)
 |-- F_Undergrad: integer (nullable = true)
 |-- P_Undergrad: integer (nullable = true)
 |-- Outstate: integer (nullable = true)
 |-- Room_Board: integer (nullable = true)
 |-- Books: integer (nullable = true)
 |-- Personal: integer (nullable = true)
 |-- PhD: integer (nullable = true)
 |-- Terminal: integer (nullable = true)
 |-- S_F_Ratio: double (nullable = true)
 |-- perc_alumni: integer (nullable = true)
 |-- Expend: integer (nullable = true)
 |-- Grad_Rate: integer (nullable = true)



In [0]:
from pyspark.ml.feature import VectorAssembler

In [24]:
data.columns

['School',
 'Private',
 'Apps',
 'Accept',
 'Enroll',
 'Top10perc',
 'Top25perc',
 'F_Undergrad',
 'P_Undergrad',
 'Outstate',
 'Room_Board',
 'Books',
 'Personal',
 'PhD',
 'Terminal',
 'S_F_Ratio',
 'perc_alumni',
 'Expend',
 'Grad_Rate']

In [0]:
assembler = VectorAssembler(inputCols=['Apps',
                                       'Accept',
                                       'Enroll',
                                       'Top10perc',
                                       'Top25perc',
                                       'F_Undergrad',
                                       'P_Undergrad',
                                       'Outstate',
                                       'Room_Board',
                                       'Books',
                                       'Personal',
                                       'PhD',
                                       'Terminal',
                                       'S_F_Ratio',
                                       'perc_alumni',
                                       'Expend',
                                       'Grad_Rate'],outputCol='features')

In [0]:
output = assembler.transform(data)

In [0]:
from pyspark.ml.feature import StringIndexer

In [0]:
indexer = StringIndexer(inputCol='Private',outputCol='PrivateIndex')

In [0]:
output_fixed = indexer.fit(output).transform(output)

In [32]:
output_fixed.printSchema()

root
 |-- School: string (nullable = true)
 |-- Private: string (nullable = true)
 |-- Apps: integer (nullable = true)
 |-- Accept: integer (nullable = true)
 |-- Enroll: integer (nullable = true)
 |-- Top10perc: integer (nullable = true)
 |-- Top25perc: integer (nullable = true)
 |-- F_Undergrad: integer (nullable = true)
 |-- P_Undergrad: integer (nullable = true)
 |-- Outstate: integer (nullable = true)
 |-- Room_Board: integer (nullable = true)
 |-- Books: integer (nullable = true)
 |-- Personal: integer (nullable = true)
 |-- PhD: integer (nullable = true)
 |-- Terminal: integer (nullable = true)
 |-- S_F_Ratio: double (nullable = true)
 |-- perc_alumni: integer (nullable = true)
 |-- Expend: integer (nullable = true)
 |-- Grad_Rate: integer (nullable = true)
 |-- features: vector (nullable = true)
 |-- PrivateIndex: double (nullable = false)



In [0]:
final_data = output_fixed.select('features','PrivateIndex')

In [0]:
train_data,test_data = final_data.randomSplit([0.7,0.3])

In [0]:
from pyspark.ml.classification import (DecisionTreeClassifier,
                                       GBTClassifier,
                                       RandomForestClassifier)

In [0]:
from pyspark.ml import Pipeline

In [0]:
dtc = DecisionTreeClassifier(labelCol='PrivateIndex')
rfc = RandomForestClassifier(labelCol='PrivateIndex',numTrees=150)
gbt = GBTClassifier(labelCol='PrivateIndex')

In [0]:
dtc_model = dtc.fit(train_data)
rfc_model = rfc.fit(train_data)
gbt_model = gbt.fit(train_data)

In [0]:
dtc_preds = dtc_model.transform(test_data)
rfc_preds = rfc_model.transform(test_data)
gbt_preds = gbt_model.transform(test_data)

In [0]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [0]:
my_bin_eval = BinaryClassificationEvaluator(labelCol='PrivateIndex')

In [44]:
print('DTC')
print(my_bin_eval.evaluate(dtc_preds))

DTC
0.950420673076923


In [53]:
print('RFC')
print(my_bin_eval.evaluate(rfc_preds))

RFC
0.9808550824175825


In [0]:
my_bin_eval2 = BinaryClassificationEvaluator(labelCol='PrivateIndex',rawPredictionCol='prediction')

In [48]:
print('GBT')
print(my_bin_eval2.evaluate(gbt_preds))

GBT
0.9132039835164836


In [0]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [0]:
acc_eval = MulticlassClassificationEvaluator(labelCol='PrivateIndex',metricName='accuracy')

In [0]:
rfc_acc = acc_eval.evaluate(rfc_preds)

In [58]:
rfc_acc

0.9349593495934959